In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import rankdata
import warnings
warnings.filterwarnings('ignore')

# 设置全局样式
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号
plt.style.use('seaborn-whitegrid')

In [ ]:
# %% [markdown]
# ## 第一部分：AHP计算主观权重

# %%
def ahp_weights(matrix):
    """
    计算AHP判断矩阵的权重并进行一致性检验
    
    参数:
    matrix -- 判断矩阵 (numpy array)
    
    返回:
    weights -- 权重向量
    ci -- 一致性指标
    cr -- 一致性比率
    """
    # 1. 列归一化
    column_sums = matrix.sum(axis=0)
    normalized = matrix / column_sums
    
    # 2. 行求和并归一化得到权重
    weights = normalized.sum(axis=1) / matrix.shape[0]
    
    # 3. 计算最大特征值 λ_max
    Aw = matrix @ weights
    lambda_max = np.mean(Aw / weights)
    
    # 4. 一致性指标 CI
    n = matrix.shape[0]
    ci = (lambda_max - n) / (n - 1)
    
    # 5. 随机一致性指标 RI (查表)
    ri_values = {1: 0, 2: 0, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45}
    ri = ri_values.get(n, 0)
    
    # 6. 一致性比率 CR
    cr = ci / ri if ri != 0 else 0
    
    return weights, ci, cr

# %%
# 专家打分的准则层判断矩阵（价格、质量、交货期、技术支持）
criteria_matrix = np.array([
    [1, 3, 5, 4],
    [1/3, 1, 3, 2],
    [1/5, 1/3, 1, 1/2],
    [1/4, 1/2, 2, 1]
])

# 显示判断矩阵
criteria_names = ['价格', '质量', '交货期', '技术支持']
print("=== AHP判断矩阵 ===")
ahp_matrix_df = pd.DataFrame(criteria_matrix, 
                           index=criteria_names, 
                           columns=criteria_names)
display(ahp_matrix_df.style.format("{:.2f}").background_gradient(cmap='Blues'))

# 计算AHP权重
ahp_weights_vec, ahp_ci, ahp_cr = ahp_weights(criteria_matrix)

# 显示结果
print("\n=== AHP权重计算结果 ===")
print(f"最大特征值 λ_max = {ahp_ci * (4-1) + 4:.3f}")
print(f"一致性指标 CI = {ahp_ci:.4f}")
print(f"一致性比率 CR = {ahp_cr:.4f} {'< 0.1，通过一致性检验！' if ahp_cr < 0.1 else '> 0.1，需调整判断矩阵！'}")

# 创建AHP权重DataFrame
ahp_results = pd.DataFrame({
    '指标': criteria_names,
    'AHP权重': ahp_weights_vec,
    '占比': [f"{w:.1%}" for w in ahp_weights_vec]
})
display(ahp_results.set_index('指标').style.format({'AHP权重': "{:.3f}"}).bar(subset=['AHP权重'], color='#5fba7d'))

In [ ]:
# %% [markdown]
# ## 第二部分：熵权法计算客观权重

# %%
def entropy_weight(data, is_positive):
    """
    计算熵权法权重
    
    参数:
    data -- 数据矩阵，行表示对象，列表示指标
    is_positive -- 布尔列表，指示各指标是否为正向指标
    
    返回:
    weights -- 熵权法计算的权重向量
    """
    # 1. 数据标准化
    n, m = data.shape  # n个对象，m个指标
    normalized = np.zeros((n, m))
    
    for j in range(m):
        if is_positive[j]:  # 正向指标
            col_min = np.min(data[:, j])
            col_max = np.max(data[:, j])
            normalized[:, j] = (data[:, j] - col_min) / (col_max - col_min)
        else:  # 负向指标
            col_min = np.min(data[:, j])
            col_max = np.max(data[:, j])
            normalized[:, j] = (col_max - data[:, j]) / (col_max - col_min)
    
    # 2. 计算比重
    p = normalized / normalized.sum(axis=0)
    
    # 3. 计算信息熵
    e = np.zeros(m)
    for j in range(m):
        # 处理p[i,j]=0的情况
        mask = p[:, j] > 0
        e[j] = -np.sum(p[mask, j] * np.log(p[mask, j])) / np.log(n)
    
    # 4. 计算信息效用值
    d = 1 - e
    
    # 5. 确定权重
    weights = d / np.sum(d)
    
    return weights, normalized

# %%
# 历史交易数据（3家供应商，4个指标）
# 行：供应商A、B、C
# 列：价格(元)、质量(分)、交货期(天)、技术支持(分)
supplier_data = np.array([
    [120, 85, 7, 90],   # 供应商A
    [105, 90, 5, 85],   # 供应商B
    [135, 80, 10, 95]   # 供应商C
])

# 指明哪些是正向指标（越大越好）
is_positive = [False, True, False, True]  # 价格(负)、质量(正)、交货期(负)、技术支持(正)

# 显示原始数据
print("\n=== 供应商历史交易数据 ===")
data_df = pd.DataFrame(supplier_data, 
                      index=['供应商A', '供应商B', '供应商C'],
                      columns=criteria_names)
display(data_df)

# 计算熵权法权重
entropy_weights_vec, normalized_data = entropy_weight(supplier_data, is_positive)

# 显示标准化数据
print("\n=== 标准化后的数据 ===")
norm_df = pd.DataFrame(normalized_data,
                      index=['供应商A', '供应商B', '供应商C'],
                      columns=criteria_names)
display(norm_df.style.format("{:.2f}"))

# 显示熵权法结果
print("\n=== 熵权法权重计算结果 ===")
# 计算中间步骤
n, m = supplier_data.shape
p = normalized_data / normalized_data.sum(axis=0)
e = np.zeros(m)
for j in range(m):
    mask = p[:, j] > 0
    e[j] = -np.sum(p[mask, j] * np.log(p[mask, j])) / np.log(n)
d = 1 - e

entropy_results = pd.DataFrame({
    '指标': criteria_names,
    '比重矩阵': [f"{np.mean(p[:,j]):.3f}" for j in range(m)],
    '信息熵(e)': [f"{e[j]:.3f}" for j in range(m)],
    '信息效用值(d)': [f"{d[j]:.3f}" for j in range(m)],
    '熵权法权重': entropy_weights_vec,
    '占比': [f"{w:.1%}" for w in entropy_weights_vec]
})
display(entropy_results.set_index('指标').style.format({
    '信息熵(e)': "{:.3f}",
    '信息效用值(d)': "{:.3f}",
    '熵权法权重': "{:.3f}"
}).bar(subset=['熵权法权重'], color='#d65f5f'))

In [ ]:
# %% [markdown]
# ## 第三部分：组合权重计算与应用

# %%
def combined_weights(ahp_weights, entropy_weights, method='linear', alpha=0.6):
    """
    计算组合权重
    
    参数:
    ahp_weights -- AHP权重向量
    entropy_weights -- 熵权法权重向量
    method -- 组合方法 ('linear' 或 'product')
    alpha -- 线性组合时AHP的权重 (0-1之间)
    
    返回:
    combined_weights -- 组合权重向量
    """
    if method == 'linear':
        # 线性组合：w = alpha * w_ahp + (1-alpha) * w_entropy
        return alpha * ahp_weights + (1 - alpha) * entropy_weights
    elif method == 'product':
        # 乘法组合：w = (w_ahp * w_entropy) / sum(w_ahp * w_entropy)
        product = ahp_weights * entropy_weights
        return product / np.sum(product)
    else:
        raise ValueError("方法必须是'linear'或'product'")

# %%
# 计算组合权重（线性组合，α=0.65）
combined_linear = combined_weights(ahp_weights_vec, entropy_weights_vec, method='linear', alpha=0.65)

# 计算组合权重（乘法组合）
combined_product = combined_weights(ahp_weights_vec, entropy_weights_vec, method='product')

# 创建权重对比表格
print("\n=== 权重对比分析 ===")
weights_comparison = pd.DataFrame({
    '指标': criteria_names,
    'AHP权重': ahp_weights_vec,
    '熵权法权重': entropy_weights_vec,
    '线性组合权重(α=0.65)': combined_linear,
    '乘法组合权重': combined_product
})

# 添加占比列
weights_comparison['线性组合占比'] = [f"{w:.1%}" for w in combined_linear]
weights_comparison['乘法组合占比'] = [f"{w:.1%}" for w in combined_product]

display(weights_comparison.set_index('指标').style.format({
    'AHP权重': "{:.3f}",
    '熵权法权重': "{:.3f}",
    '线性组合权重(α=0.65)': "{:.3f}",
    '乘法组合权重': "{:.3f}"
}).background_gradient(subset=['线性组合权重(α=0.65)', '乘法组合权重'], cmap='viridis'))

In [ ]:
# 可视化权重对比
plt.figure(figsize=(12, 6))
x = np.arange(len(criteria_names))
width = 0.2

plt.bar(x - width*1.5, ahp_weights_vec, width, label='AHP权重', color='#1f77b4')
plt.bar(x - width/2, entropy_weights_vec, width, label='熵权法权重', color='#ff7f0e')
plt.bar(x + width/2, combined_linear, width, label='线性组合权重(α=0.65)', color='#2ca02c')
plt.bar(x + width*1.5, combined_product, width, label='乘法组合权重', color='#d62728')

plt.xlabel('评价指标', fontsize=12)
plt.ylabel('权重值', fontsize=12)
plt.title('不同方法计算的权重对比', fontsize=14, fontweight='bold')
plt.xticks(x, criteria_names)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 在柱子上显示具体数值
for i, v in enumerate(ahp_weights_vec):
    plt.text(i - width*1.5, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
for i, v in enumerate(entropy_weights_vec):
    plt.text(i - width/2, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
for i, v in enumerate(combined_linear):
    plt.text(i + width/2, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
for i, v in enumerate(combined_product):
    plt.text(i + width*1.5, v + 0.01, f'{v:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('weight_comparison.png', dpi=300)
plt.show()

In [ ]:
# %% [markdown]
# ## 第四部分：使用组合权重进行供应商评估

# %%
def evaluate_suppliers(normalized_data, weights):
    """
    使用权重评估供应商
    
    参数:
    normalized_data -- 标准化后的数据矩阵
    weights -- 权重向量
    
    返回:
    scores -- 供应商得分
    rankings -- 供应商排名
    """
    # 计算加权得分
    scores = np.dot(normalized_data, weights)
    
    # 获取排名（从高到低）
    rankings = rankdata(-scores, method='min')
    
    return scores, rankings

# %%
# 使用线性组合权重评估供应商
linear_scores, linear_rankings = evaluate_suppliers(normalized_data, combined_linear)

# 使用乘法组合权重评估供应商
product_scores, product_rankings = evaluate_suppliers(normalized_data, combined_product)

# 创建评估结果表格
print("\n=== 供应商评估结果 ===")
evaluation_results = pd.DataFrame({
    '供应商': ['供应商A', '供应商B', '供应商C'],
    '线性组合得分': linear_scores,
    '线性组合排名': linear_rankings,
    '乘法组合得分': product_scores,
    '乘法组合排名': product_rankings
})

# 添加排名描述
evaluation_results['线性组合评价'] = evaluation_results['线性组合排名'].map({
    1: '最佳选择', 
    2: '次优选择', 
    3: '需改进'
})
evaluation_results['乘法组合评价'] = evaluation_results['乘法组合排名'].map({
    1: '最佳选择', 
    2: '次优选择', 
    3: '需改进'
})

# 按线性组合得分排序
evaluation_results = evaluation_results.sort_values('线性组合得分', ascending=False)

# 显示结果
display(evaluation_results.style.format({
    '线性组合得分': "{:.4f}",
    '乘法组合得分': "{:.4f}"
}).apply(lambda x: ['background: lightgreen' if v == 1 else '' for v in x['线性组合排名']], axis=1, subset=['线性组合排名'])
  .apply(lambda x: ['background: lightgreen' if v == 1 else '' for v in x['乘法组合排名']], axis=1, subset=['乘法组合排名']))

# 可视化供应商得分
plt.figure(figsize=(12, 6))
x = np.arange(3)
width = 0.35

plt.bar(x - width/2, linear_scores, width, label='线性组合得分', color='#1f77b4')
plt.bar(x + width/2, product_scores, width, label='乘法组合得分', color='#ff7f0e')

plt.xlabel('供应商', fontsize=12)
plt.ylabel('综合得分', fontsize=12)
plt.title('供应商综合评估得分对比', fontsize=14, fontweight='bold')
plt.xticks(x, ['供应商A', '供应商B', '供应商C'])
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 在柱子上显示具体数值和排名
for i, (v1, v2) in enumerate(zip(linear_scores, product_scores)):
    plt.text(i - width/2, v1 + 0.01, f'得分:{v1:.4f}\n排名:{linear_rankings[i]}', 
             ha='center', va='bottom', fontsize=9)
    plt.text(i + width/2, v2 + 0.01, f'得分:{v2:.4f}\n排名:{product_rankings[i]}', 
             ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('supplier_evaluation.png', dpi=300)
plt.show()

In [ ]:
# %% [markdown]
# ## 第五部分：结果分析与决策建议

# %%
# 分析AHP与熵权法的差异
print("\n=== AHP与熵权法权重差异分析 ===")
diff_analysis = pd.DataFrame({
    '指标': criteria_names,
    'AHP权重': ahp_weights_vec,
    '熵权法权重': entropy_weights_vec,
    '差异值': entropy_weights_vec - ahp_weights_vec,
    '差异率': (entropy_weights_vec - ahp_weights_vec) / ahp_weights_vec * 100
})

# 标记差异显著的指标
diff_analysis['分析'] = diff_analysis.apply(lambda row: 
    'AHP高估' if row['差异值'] < -0.1 else 
    'AHP低估' if row['差异值'] > 0.1 else 
    '基本一致', axis=1)

display(diff_analysis.style.format({
    'AHP权重': "{:.3f}",
    '熵权法权重': "{:.3f}",
    '差异值': "{:.3f}",
    '差异率': "{:.1f}%"
}).apply(lambda x: ['background-color: #ffcccc' if v < -0.1 else 
                    'background-color: #ccffcc' if v > 0.1 else 
                    '' for v in x['差异值']], axis=1, subset=['差异值']))

# %%
# 决策建议
print("\n=== 最终决策建议 ===")

# 确定最佳供应商
best_supplier_linear = evaluation_results.iloc[0]['供应商']
best_score_linear = evaluation_results.iloc[0]['线性组合得分']
best_supplier_product = evaluation_results.iloc[0]['供应商']
best_score_product = evaluation_results.iloc[0]['乘法组合得分']

# 分析关键指标
key_indices = np.argsort(-np.abs(entropy_weights_vec - ahp_weights_vec))[:2]
key_factors = [criteria_names[i] for i in key_indices]

print(f"1. **推荐选择：{best_supplier_linear}** (线性组合方法)")
print(f"   - 综合得分: {best_score_linear:.4f} (领先第二名 {best_score_linear - evaluation_results.iloc[1]['线性组合得分']:.4f})")
print(f"   - 选择依据: 线性组合方法(α=0.65)更符合企业'战略导向为主，兼顾数据客观性'的原则")

print(f"\n2. **关键因素分析**:")
print(f"   - {key_factors[0]}: AHP权重与熵权法权重差异最大，表明该指标在战略重要性与实际区分能力上存在明显分歧")
print(f"   - {key_factors[1]}: 同样存在显著差异，需要重点关注")

print("\n3. **AHP与熵权法差异解读**:")
if diff_analysis['差异值'].max() > 0.1:
    print(f"   - 企业战略认为不太重要的指标（如{diff_analysis.loc[diff_analysis['差异值'] > 0.1, '指标'].values[0]}），")
    print("     实际数据表明其区分能力很强，可能是潜在风险点")
if diff_analysis['差异值'].min() < -0.1:
    print(f"   - 企业战略高度重视的指标（如{diff_analysis.loc[diff_analysis['差异值'] < -0.1, '指标'].values[0]}），")
    print("     实际数据表明其区分能力有限，可能过度投入资源")

print("\n4. **管理建议**:")
print("   - 与首选供应商签订合同时，重点关注差异显著的指标，设置明确的KPI")
print("   - 建立季度评估机制，动态调整α值（市场稳定期α可提高，波动期α应降低）")
print("   - 对差异显著的指标开展专项分析，确定是调整战略重点还是改进数据收集方法")

# %%
# %% [markdown]
# ## 附录：敏感性分析（α值变化对结果的影响）

# %%
# 分析不同α值对结果的影响
alpha_values = np.linspace(0, 1, 11)
supplier_scores = np.zeros((3, len(alpha_values)))

for i, alpha in enumerate(alpha_values):
    weights = combined_weights(ahp_weights_vec, entropy_weights_vec, method='linear', alpha=alpha)
    scores, _ = evaluate_suppliers(normalized_data, weights)
    supplier_scores[:, i] = scores

# 创建敏感性分析表格
print("\n=== 敏感性分析：α值变化对供应商得分的影响 ===")
sensitivity_df = pd.DataFrame({
    'α值': alpha_values,
    '供应商A得分': supplier_scores[0, :],
    '供应商B得分': supplier_scores[1, :],
    '供应商C得分': supplier_scores[2, :]
})

# 确定每个α值下的最佳供应商
sensitivity_df['最佳供应商'] = sensitivity_df.apply(
    lambda row: ['供应商A', '供应商B', '供应商C'][np.argmax([row['供应商A得分'], row['供应商B得分'], row['供应商C得分']])], 
    axis=1
)

display(sensitivity_df.style.format({
    'α值': "{:.2f}",
    '供应商A得分': "{:.4f}",
    '供应商B得分': "{:.4f}",
    '供应商C得分': "{:.4f}"
}).apply(lambda x: ['background: lightgreen' if v == x['最佳供应商'] else '' 
                    for v in ['供应商A', '供应商B', '供应商C']], 
         axis=1, 
         subset=['最佳供应商']))

# 绘制敏感性分析图
plt.figure(figsize=(12, 7))
for i, supplier in enumerate(['供应商A', '供应商B', '供应商C']):
    plt.plot(alpha_values, supplier_scores[i, :], 'o-', linewidth=2, markersize=6, label=supplier)

# 标记决策转折点
crossover_points = []
for i in range(len(alpha_values)-1):
    if (supplier_scores[0,i] > supplier_scores[1,i] and supplier_scores[0,i+1] < supplier_scores[1,i+1]) or \
       (supplier_scores[0,i] < supplier_scores[1,i] and supplier_scores[0,i+1] > supplier_scores[1,i+1]):
        crossover_points.append(alpha_values[i])
    if (supplier_scores[0,i] > supplier_scores[2,i] and supplier_scores[0,i+1] < supplier_scores[2,i+1]) or \
       (supplier_scores[0,i] < supplier_scores[2,i] and supplier_scores[0,i+1] > supplier_scores[2,i+1]):
        crossover_points.append(alpha_values[i])
    if (supplier_scores[1,i] > supplier_scores[2,i] and supplier_scores[1,i+1] < supplier_scores[2,i+1]) or \
       (supplier_scores[1,i] < supplier_scores[2,i] and supplier_scores[1,i+1] > supplier_scores[2,i+1]):
        crossover_points.append(alpha_values[i])

if crossover_points:
    for point in set(crossover_points):
        plt.axvline(x=point, color='gray', linestyle='--', alpha=0.7)
        plt.text(point+0.01, 0.95, f'转折点 α={point:.2f}', 
                 transform=plt.gca().get_xaxis_transform(), 
                 fontsize=9)

plt.xlabel('α值 (AHP权重占比)', fontsize=12)
plt.ylabel('综合得分', fontsize=12)
plt.title('敏感性分析：α值变化对供应商评估结果的影响', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.ylim(0, max(np.max(supplier_scores), 0.95))

plt.tight_layout()
plt.savefig('sensitivity_analysis.png', dpi=300)
plt.show()

print("\n敏感性分析结论:")
print(f"- 当α ∈ [0.00, {crossover_points[0]:.2f}) 时，最佳供应商为 供应商C")
print(f"- 当α ∈ [{crossover_points[0]:.2f}, {crossover_points[1]:.2f}) 时，最佳供应商为 {best_supplier_linear}")
print(f"- 当α ∈ [{crossover_points[1]:.2f}, 1.00] 时，最佳供应商为 供应商B")
print(f"\n推荐α值范围: [{crossover_points[0]:.2f}, {crossover_points[1]:.2f}]，当前选择α=0.65在此范围内")